# Quality indicator data: hospitals by location and department with disease description



## Install deepl module

`deepl` module will be used to translate the features to english: https://www.deepl.com/pro-api?cta=header-pro-api/

In [ ]:
!pip install selenium
!pip install --upgrade deepl
import deepl
! pip install streamlit
import streamlit as st
 

## Import necessary modules:

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import time
from pymongo import collection
from enum import Enum
import plotly.express as px

## Import data in desired format:

In [ ]:
df = pd.read_csv("src/raw/quality_indicators.csv",
                    encoding = "ISO-8859-1",
                    sep="\n",
                    nrows=50)
df = df.iloc[ : , 0].str.split(';', expand=True)


In [ ]:
df

,0,1,2,3,4,5,6,7,8,9
0,Les Hôpitaux Universitaires de Genève HUG,A Maladies cardiaques,,,,,,,,
1,Les Hôpitaux Universitaires de Genève HUG,A.1 Infarctus du myocarde,,,,,,,,
2,Les Hôpitaux Universitaires de Genève HUG,"A.1.1.M DP infarctus du myocarde (âge >19), mo...",5.3%,5.3%,1,2'849,4.8%,4.6%,1,621
3,Les Hôpitaux Universitaires de Genève HUG,"A.1.2.M DP infarctus du myocarde, âge 20-44, m...",1.6%,1.2%,1.4,188,0.0%,0.1%,0,37
4,Les Hôpitaux Universitaires de Genève HUG,"A.1.3.M DP infarctus du myocarde, âge 45-64, m...",2.4%,1.7%,1.4,1'059,3.5%,1.6%,2.2,227
5,Les Hôpitaux Universitaires de Genève HUG,"A.1.4.M DP infarctus du myocarde, âge 65-84, m...",5.7%,5.2%,1.1,1'158,4.8%,5.0%,1,271
6,Les Hôpitaux Universitaires de Genève HUG,"A.1.5.M DP infarctus du myocarde, âge >84, mor...",12.8%,16.3%,0.8,444,10.5%,13.4%,0.8,86
7,Les Hôpitaux Universitaires de Genève HUG,"A.1.7.M DP infarctus du myocarde (âge >19), ad...",5.5%,6.8%,0.8,2'491,5.2%,6.2%,0.8,559
8,Les Hôpitaux Universitaires de Genève HUG,"A.1.8.M DP infarctus du myocarde (âge >19), tr...",5.3%,4.5%,1.2,243,3.6%,3.6%,1,28
9,Les Hôpitaux Universitaires de Genève HUG,A.1.14.P DP infarctus du myocarde avec cathété...,84.5%,62.2%,*,2'408,84.7%,64.6%,*,526


## Data pre-processing functions:

In [ ]:
def read_table(csv_path: str, nrows=None, col_names=None) -> pd.DataFrame:
    '''
    Read csv file into dataframe with the correct encoding and separator
    Args:
        csv_path: raw csv file with semicolon and new line separators

    Returns:
        pd.DataFrame: dataframe containing names of hospitals, disease groups with description,
         disease code, mortality, transfer rate, length of hospital stay
    '''
    df = pd.read_csv(csv_path,
                    encoding = "ISO-8859-1",
                    sep="\n",
                    nrows=nrows)
    df = df.iloc[ : , 0].str.split(';', expand=True)

    if col_names:
      raw_cols = df.columns
      raw_cols[ : len(col_names)] = col_names
      df.columns = raw_cols

    return(df)


def load_vocabulary(path_to_vocabulary: str) -> json:
    '''
    Load already created json file to map terms in various languages to English
    Args:
      path_to_vocabulary: path to json file

    Returns:
      json: vocabulary of English terms in json format
    '''
    with open(path_to_vocabulary, "r") as read_file:
      vocabulary = json.load(read_file)
    return(vocabulary)


def create_vocabulary_deepL(terms: pd.Series, filename=None, log=False) -> dict:
    '''
    Translate terms to english and save to json file if
    filename not None
    Args:
      terms: iterable of strings to translate
      filename: save dictionary of translated terms to filepath
      log: write log

    Returns:
      dict: dictionary of phrase in the original language and the translation
    '''
    # auth_key - input your deepl translator key 
    translator = deepl.Translator(auth_key)
    translations = {}
    unique_elements = terms.unique()
    if log:
        print(f'length of unique:{len(unique_elements)}')
        start = time.time()
    for index, element in enumerate(unique_elements):
        translations[element] = translator.translate_text(element, target_lang="EN-GB").text
        if log:
            if index % 100 == 0:
                end = time.time()
                print(f'done {index} elements and time {end - start}')
    if filename != None:
        a_file = open(filename, "w")
        json.dump(translations, a_file)
        a_file.close()
    return (translations)


def translate_terms(terms: pd.Series, load_vocabulary=False, vocabulary_filepath=None) -> dict:
    '''
    Translate an input list of string to english.
    Args: 
        terms: iterable of terms to translate
        load_vocabulary: if true, load json file of already translated terms
        vocabulary_filepath: path to json file to load/save the file from/to
    
    Returns:
        dict: dictionary of terms translated to english
    '''
    if load_vocabulary:
      translations = load_vocabulary(vocabulary_filepath)
    else:
      translations = create_vocabulary_deepL(
          terms,
          filename=vocabulary_filepath
      )
    terms_eng = [translations[element] for element in terms]
    return(terms_eng)


def extract_indicator_code(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Extract disease code and place it in new column
    Args:
        df: dataframe containing names of hospitals, disease groups with description,
         disease code, mortality, transfer rate, length of hospital stay

    Returns:
        pd.DataFrame: dataframe containing separate indicator code column
    '''
    df["code"] = df["indicator_eng"].str.extract('([^\s]+)', expand=True)
    print(f"dim is:{df.shape}")
    return df


def extract_legend(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Retrieve code legend from main dataframe
    Args:
        df: dataframe containing names of hospitals, disease groups with description,
         disease code, mortality, transfer rate, length of hospital stay

    Returns:
        pd.DataFrame: new dataframe containing only code legend
    '''
    code_legend = df[
                    df['code'].str.len() <= 3][
                     ['code', 'indicator_eng']
                    ].drop_duplicates()
    return code_legend


def extract_mortality(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Extract mortality information
    Args:
        df: dataframe containing names of hospitals, disease groups with description,
         disease code, mortality, transfer rate, length of hospital stay

    Returns:
        pd.DataFrame: dataframe with a separate mortality column
    '''
    mortality_codes = df[df['code'].str.endswith(".M")].code.unique()
    df['mortality'] = df['code'].isin(mortality_codes)
    return df


def extract_age(input_string: str) -> str: 
    '''
    Extract age information into a separate string
    Args:
        input_string: string containing disease code, disease description, 
        and age groups with various separators

    Returns:
        str: string containing separate age groups in unified format
    '''
    match = re.search('(?<=\Wage)(.*)', input_string)
    new_string = "None"
    if match:
        placeholder = f"age {match.group(1)[:10]}"
        match = re.search('(?<=age)(\D*)(\d*)(-)(\d*)', placeholder)
        if match:
            new_string = f'{match.group(2)}-{match.group(4)}'
        else:
            match = re.search('(?<=age)(\D*)(\d*)', placeholder)
            if match:
                if ">" in match.group(1):
                    new_string = f'{match.group(2)}-100'
                elif "<" in match.group(1):
                    new_string = f'0-{match.group(2)}'
    return (new_string)


def remove_length_stay(df: pd.DataFrame, length_stay_codes=None) -> pd.DataFrame:
    '''
    Extract length of stay information
    Args:
        df: dataframe containing names of hospitals, disease groups with description,
         disease code, mortality, transfer rate, length of hospital stay

    Returns:
        pd.DataFrame: dataframe without length of stay information
    '''
    if length_stay_codes == None:
      length_stay_codes = [
        'A.1.12.X', 'B.1.15.X', 'E.1.1.X',
        'E.2.1.X', 'F.1.5.X', 'G.3.1.X',
        'H.4.5.X', 'I.1.23.X', 'I.1.24.X',
        'Z.1.1.X'
        ]
    df = df[~df['code'].isin(length_stay_codes)]
    return df


def remove_transfer(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Extract transfer information
    Args:
        df: dataframe containing names of hospitals, disease groups with description,
         disease code, mortality, transfer rate

    Returns:
        pd.DataFrame: dataframe without transfer information
    '''
    df = df[~df['code'].str.endswith(".V")]
    return df


def remove_unuseful_info(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Clean the inputted dataframe from unuseful data - drop columns with all NaN, remove rows without data,
    remove code legend from the dataframe.
    Args:
        df: dataframe containing names of hospitals, disease groups with description,
         disease code, mortality, transfer rate, length of hospital stay

    Returns:
        pd.DataFrame: clean dataframe containing only useful values
    '''
    df.dropna(how='all', axis=1, inplace=True)
    df = df.replace([np.nan, '*', '-', ' ', ''], "None")
    instituzione_row_index = df[df.hospital.str.contains('[I-i]nstitu')].index
    indicator_row_index = df[df.indicator.isin(['indicator', 'indicatore'])].index
    index_to_drop = instituzione_row_index.append(indicator_row_index).unique()
    df = df.drop(index_to_drop)
    code_legend = df[df['code'].str.len() <= 3][['code', 'indicator_eng']].drop_duplicates()
    df = df[~df['code'].isin(code_legend.code)]
    df.replace(regex = ["\\'"], value='', inplace=True)
    def remove_perc(x):
      return(x.astype(str).str.replace('%', ''))
    return df.apply(remove_perc)


def separate_address(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Create separate columns for house number, street, ZIP code and city
    Args:
        df: dataframe containing names of hospitals, disease groups with description,
         disease code, mortality, transfer rate, length of hospital stay

    Returns:
        pd.DataFrame: dataframe with separate columns for address items
    '''
    try:
      sep_interim_df = df["hospital"].str.split(",", expand=True)
      hospital = sep_interim_df[0]
      street_number = sep_interim_df[1]
      ZIP_city = sep_interim_df[2].str.extract('(?P<ZIP>\d+)(?P<City>.*)', expand=True)
      final_df = pd.concat([ZIP_city, df], axis=1)
      final_df['hospital'] = hospital
      final_df.insert(2, 'street_number', street_number)
      final_df.reset_index(inplace=True)
      final_df.drop(["index"], axis=1, inplace=True)
      return(final_df)
    except:
      for col in ['street_number', "ZIP","City"]:
        df[col] = "None"
      return df


def data_wrangling(df: pd.DataFrame) -> pd.DataFrame:
    '''
    Use data preprocessing functions to obtain clean dataframe
    Args:
        df: raw dataframe obtained from initial csv file

    Returns:
        pd.DataFrame: clean data frame with names of hospitals, disease groups with description,
         disease code, mortality, transfer rate, length of hospital stay
    '''
    cols_eng = translate_terms(df.columns)
    indicator_eng = translate_terms(df.indicator)
    df.columns = cols_eng
    df.columns = [
          col_name.replace(" ", "_") for col_name in df.columns
          ]
    df['indicator_eng'] = indicator_eng
    df = extract_indicator_code(df)
    df_legend = extract_legend(df)
    clean_df = df.copy()
    clean_df['age_cat'] = clean_df['indicator_eng'].apply(extract_age)
    clean_df = extract_mortality(clean_df)
    clean_df = remove_length_stay(clean_df)
    clean_df = remove_transfer(clean_df)
    clean_df = separate_address(clean_df)
    clean_df = remove_unuseful_info(clean_df)
    clean_df = clean_df.reset_index().drop(["index"], axis=1)
    return(clean_df)


In [ ]:
# set col names for table_1
col_names_quality_indic = [
  "hospital", 
  "indicator", 
  "observed_rate_2014-2018_%",	
  "expected_rate_2014-2018_%", 
  "therapeutic_value_2014-2018", 
  "number_of_cases_2014-2018", 
  "observed_rate_2019_%",	
  "expected_rate_2019_%", 
  "therapeutic_value_2019", 
  "number_of_cases_2019",
  "None"
  ]



table_qual_indic = read_table(
    csv_path="src/raw/quality_indicators.csv")
# Drop 10th column above 50 rows

In [ ]:
# rename cols
table_qual_indic.columns = col_names_quality_indic
# run pipeline

table_2_w = data_wrangling(table_qual_indic)

# Exploratory data analysis

In [ ]:
import pandas as pd
import numpy as np
import json
import plotly.express as px

## Dataset 2019 based on location:

In [ ]:
def merge_hospital_cantons(df):
  df = df[~df['number_of_cases_2019'].str.contains('None')]
  df['hospital'] = df['hospital'].str.split(' - ').apply(lambda x: x[0].strip())
  df['main_code'] = df['code'].astype(str).str[0]
  disease_group_dict = {'A': 'Cardiac diseases',
                        'B': 'Diseases of the nervous system, cerebrovascular accident (stroke)',
                        'C': 'Geriatric Medicine',
                        'D': 'Lung diseases',
                        'E': 'Diseases of the abdominal organs',
                        'F': 'Vascular Diseases',
                        'G': 'Gynecology and obstetrics',
                        'H': 'Diseases of the urinary tract and male genitalia',
                        'I': 'Diseases of the bones, joints, connective tissues',
                        'J': 'Complex conditions',
                        'K': 'Skin disorders',
                        'L': 'Highly specialized medicine',
                        'M': 'Palliative Medicine',
                        }
  df['disease_group'] = df['main_code'].map(disease_group_dict)
  df = df.astype({'number_of_cases_2014-2018' : 'int'})
  df = df.astype({'number_of_cases_2019' : 'int'})
  sum_column = df["number_of_cases_2014-2018"] + df["number_of_cases_2019"]
  df['number_of_cases_2014_2019'] = sum_column

  hospit_city_canton = pd.read_csv(
      'src/processed/hospitals_with_city_canton.csv', sep=';'
      )
  hospit_city_canton = hospit_city_canton.merge(df, on="hospital")
  hospit_city_canton.drop(['Unnamed: 0_x', 
                           'canton_english', 
                           'Unnamed: 0_y', 
                           'indicator', 
                           'therapeutic_value_2014-2018', 
                           'therapeutic_value_2019', 
                           'None'], 
                            axis=1, inplace=True)
  hospit_city_canton = hospit_city_canton[[
                                           'hospital', 'city', 'canton_name', 
                                           'main_code', 'disease_group', 'code', 
                                           'age_cat', 'mortality', 'indicator_eng', 
                                           'number_of_cases_2014_2019', 
                                           'number_of_cases_2014-2018', 
                                           'number_of_cases_2019',
                                            'observed_rate_2014-2018_%',	
                                           'expected_rate_2014-2018_%',	
                                           'observed_rate_2019_%',	
                                            'expected_rate_2019_%']]
  hospit_city_canton_disease_group = hospit_city_canton.astype(
        {'number_of_cases_2019' : 'int'}).groupby([
                                                   'hospital', 'city', 
                                                   'canton_name', 'main_code', 
                                                   'disease_group'])['number_of_cases_2014_2019'].agg(['sum'])
  hospit_city_canton_disease_group.to_csv(
      'src/processed/hospital_canton_group.csv'
      )
  quality_indic_disease_sum = pd.read_csv(
      'src/processed/processed_data/hospital_canton_group.csv'
      )
  quality_indic_disease_sum.rename(columns={"sum": "number_of_cases_2014_2019"}, inplace=True)
  quality_indic_disease_sum = quality_indic_disease_sum[
                                                        quality_indic_disease_sum['number_of_cases_2014_2019'] != 0]
  quality_indic_disease_sum.reset_index(inplace=True)
  quality_indic_disease_sum.drop(['index'], axis=1, inplace=True)
  return quality_indic_disease_sum

In [ ]:
read_qual2_2019 = pd.read_csv('src/processed/processed_quality_indicators.csv')

In [ ]:
hospital_cantons = merge_hospital_cantons(read_qual2_2019)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

In [ ]:
hospital_cantons

,hospital,city,canton_name,main_code,disease_group,number_of_cases_2014_2019
0,Adus Medica AG,Dielsdorf,Zürich,E,Diseases of the abdominal organs,1266
1,Adus Medica AG,Dielsdorf,Zürich,F,Vascular Diseases,2
2,Adus Medica AG,Dielsdorf,Zürich,G,Gynecology and obstetrics,31
3,Adus Medica AG,Dielsdorf,Zürich,H,Diseases of the urinary tract and male genitalia,4
4,Adus Medica AG,Dielsdorf,Zürich,I,"Diseases of the bones, joints, connective tissues",131
...,...,...,...,...,...,...
1497,Zuger Kantonsspital AG,Baar,Zug,H,Diseases of the urinary tract and male genitalia,4557
1498,Zuger Kantonsspital AG,Baar,Zug,I,"Diseases of the bones, joints, connective tissues",3529
1499,Zuger Kantonsspital AG,Baar,Zug,J,Complex conditions,3350
1500,Zuger Kantonsspital AG,Baar,Zug,K,Skin disorders,67


In [ ]:
canton_check = hospital_cantons.canton_name.drop_duplicates()

In [ ]:
canton_check.reset_index()

,index,canton_name
0,0,Zürich
1,6,Zug
2,17,Aargau
3,39,Appenzell Ausserrhoden
4,48,Basel-stadt
5,59,Vaud
6,72,Ticino
7,82,Graubünden
8,164,Valais
9,173,Genève


# Plotting according to:

## Disease number per disease group in cantons

1. Without G

In [ ]:
hospital_cantons

,hospital,city,canton_name,main_code,disease_group,number_of_cases_2014_2019
0,Adus Medica AG,Dielsdorf,Zürich,E,Diseases of the abdominal organs,1266
1,Adus Medica AG,Dielsdorf,Zürich,F,Vascular Diseases,2
2,Adus Medica AG,Dielsdorf,Zürich,G,Gynecology and obstetrics,31
3,Adus Medica AG,Dielsdorf,Zürich,H,Diseases of the urinary tract and male genitalia,4
4,Adus Medica AG,Dielsdorf,Zürich,I,"Diseases of the bones, joints, connective tissues",131
...,...,...,...,...,...,...
1497,Zuger Kantonsspital AG,Baar,Zug,H,Diseases of the urinary tract and male genitalia,4557
1498,Zuger Kantonsspital AG,Baar,Zug,I,"Diseases of the bones, joints, connective tissues",3529
1499,Zuger Kantonsspital AG,Baar,Zug,J,Complex conditions,3350
1500,Zuger Kantonsspital AG,Baar,Zug,K,Skin disorders,67


In [ ]:
group_disease_cantons_wo_G = hospital_cantons[~hospital_cantons['main_code'].str.contains('G')]

In [ ]:
group_disease_cantons_wo_G = group_disease_cantons_wo_G.astype(
        {'number_of_cases_2014_2019' : 'int'}).groupby(['canton_name', 'main_code', 'disease_group']).agg({'number_of_cases_2014_2019':'max'})

In [ ]:
group_disease_cantons_wo_G.reset_index(inplace=True)

In [ ]:
group_disease_cantons_wo_G = group_disease_cantons_wo_G.sort_values('number_of_cases_2014_2019', ascending=False).drop_duplicates(['canton_name'])
group_disease_cantons_wo_G

,canton_name,main_code,disease_group,number_of_cases_2014_2019
56,Bern,A,Cardiac diseases,168902
280,Zürich,A,Cardiac diseases,119576
44,Basel-stadt,A,Cardiac diseases,111792
257,Vaud,A,Cardiac diseases,82536
200,St. Gallen,A,Cardiac diseases,79363
223,Ticino,A,Cardiac diseases,78430
79,Genève,A,Cardiac diseases,77693
124,Luzern,A,Cardiac diseases,72441
0,Aargau,A,Cardiac diseases,60110
245,Valais,A,Cardiac diseases,47148


In [ ]:
fig = px.histogram(group_disease_cantons_wo_G, x="canton_name", y="number_of_cases_2014_2019", color="disease_group", log_x=False, width=1500, height=1000)
fig.show()

2. With G

In [ ]:
group_diseases_cantons_g = hospital_cantons.astype(
        {'number_of_cases_2014_2019' : 'int'}).groupby(['canton_name', 'main_code', 'disease_group']).agg({'number_of_cases_2014_2019':'max'})

In [ ]:
group_diseases_cantons_g

number_of_cases_2014_2019
canton_name main_code disease_group                                                                
Aargau      A         Cardiac diseases                                                        60110
            B         Diseases of the nervous system, cerebrovascular...                      30056
            C         Geriatric Medicine                                                       6631
            D         Lung diseases                                                           13542
            E         Diseases of the abdominal organs                                        15259
...                                                                                             ...
Zürich      I         Diseases of the bones, joints, connective tissues                       28179
            J         Complex conditions                                                      37809
            K         Skin disorders                                                           3367
            L         Highly specialized medicine                                              4184
            M         Palliative Medicine                                                      2486

[318 rows x 1 columns]

In [ ]:
group_diseases_cantons_g.reset_index(inplace=True)

In [ ]:
most_pop_disease_canton = group_diseases_cantons_g.sort_values('number_of_cases_2014_2019', ascending=False).drop_duplicates(['canton_name'])
most_pop_disease_canton

,canton_name,main_code,disease_group,number_of_cases_2014_2019
61,Bern,A,Cardiac diseases,168902
305,Zürich,A,Cardiac diseases,119576
48,Basel-stadt,A,Cardiac diseases,111792
92,Genève,G,Gynecology and obstetrics,87714
280,Vaud,A,Cardiac diseases,82536
141,Luzern,G,Gynecology and obstetrics,79503
218,St. Gallen,A,Cardiac diseases,79363
243,Ticino,A,Cardiac diseases,78430
0,Aargau,A,Cardiac diseases,60110
237,Thurgau,G,Gynecology and obstetrics,56286


In [ ]:
fig = px.histogram(most_pop_disease_canton, x="canton_name", y="number_of_cases_2014_2019", color="disease_group", log_x=False, width=1500, height=1000)
fig.show()

## 3. Choropleth map containing cantons with hospitals and numbers of diseases:

In [ ]:
lon_lat_quality_df = pd.read_csv('src/processed/lon_lat_quality_df.csv')

In [ ]:
lon_lat_quality_df

,Unnamed: 0,lat,lng,population,population_proper,city,canton_name,hospital,number_of_cases
0,0,47.3786,8.5400,434008.0,434008.0,Zürich,Zürich,Geburtshaus Delphys,2418
1,1,47.3786,8.5400,434008.0,434008.0,Zürich,Zürich,Klinik Hirslanden AG,189222
2,2,47.3786,8.5400,434008.0,434008.0,Zürich,Zürich,Klinik Im Park,70258
3,3,47.3786,8.5400,434008.0,434008.0,Zürich,Zürich,Klinik Pyramide am See AG,8488
4,4,47.3786,8.5400,434008.0,434008.0,Zürich,Zürich,Limmatklinik AG,5004
...,...,...,...,...,...,...,...,...,...
129,129,46.5336,9.8719,2924.0,2924.0,Samedan,Graubünden,Spital Oberengadin,11491
130,130,47.4012,8.2153,2897.0,2897.0,Othmarsingen,Aargau,Geburtshus Storchenäscht AG,1944
131,131,46.9664,9.6833,2679.0,2679.0,Schiers,Graubünden,Flury Stiftung Spital Schiers,11939
132,132,47.5819,8.2194,2169.0,2169.0,Leuggern,Aargau,Asana Spital Leuggern AG,22792


In [ ]:
with open("src/raw/georef-switzerland-kanton.geojson") as response:
    cantons = json.load(response)

cantons["features"][0]["properties"]

{'geo_point_2d': [46.2204844659, 6.13300925662],
 'kan_area_code': 'CHE',
 'kan_code': '25',
 'kan_name': 'Genève',
 'kan_type': 'Kanton',
 'year': '2020'}

In [ ]:
 fig = px.scatter_mapbox(
    lon_lat_quality_df, 
    color="number_of_cases",
    size='number_of_cases',
    lat='lat', lon='lng', 
    center={"lat": 46.8, "lon": 8.3},
    hover_data=['hospital', 'number_of_cases', 'city', 'canton_name'],
    mapbox_style="open-street-map", 
    zoom=6.3,
    opacity=0.8,
    width=900,
    height=500,
    labels={"canton_name":"Canton",
            "hospital": "Hospital",
            "city": "City",
            "number_of_cases":"Number of cases"},
    title="<b>Number of cases 2014-2019 per hospital</b>",
    color_continuous_scale="Viridis"
)
fig.update_layout(margin={"r":0,"t":35,"l":0,"b":0},
                  font={"family":"Sans",
                       "color":"maroon"},
                  hoverlabel={"bgcolor":"white", 
                              "font_size":15,
                             "font_family":"Arial"},
                  title={"font_size":20,
                        "xanchor":"left", "x":0.01,
                        "yanchor":"bottom", "y":0.95}
                 )
fig.show()